In [20]:
from read_data import *
from hyper import *
from alignment import *
from datasets.preprecossing import *
import rpy2

In [21]:
rpy2.__version__

'3.5.13'

In [31]:
from rpy2.robjects.packages import importr
import rpy2.robjects as robjects

Spatialpc  = importr('SpatialPCA')
ggplot2 = importr('ggplot2')
ggplot2
a = robjects.r('a<-c(1,2,3)')
print(a)

None


In [46]:
robjects.r("load(paste0('./datas/pca/DLPFC/LIBD_sample1.RData'))")
robjects.r("xy_coords=as.matrix(xy_coords)")
robjects.r("rownames(xy_coords) = colnames(count_sub)");
robjects.r('LIBD = CreateSpatialPCAObject(counts=count_sub, location=xy_coords, project = "SpatialPCA",gene.type="spatial",sparkversion="spark",numCores_spark=5,gene.number=3000, customGenelist=NULL,min.loctions = 20, min.features=20)')
robjects.r('LIBD = SpatialPCA_buildKernel(LIBD, kerneltype="gaussian", bandwidthtype="SJ",bandwidth.set.by.user=NULL)')
robjects.r('LIBD = SpatialPCA_EstimateLoading(LIBD,fast=FALSE,SpatialPCnum=20) ')
robjects.r('LIBD=SpatialPCA_SpatialPCs(LIBD, fast=FALSE)')
# robjects.r('LIBD=SpatialPCA_SpatialPCs(LIBD, fast=FALSE)')


## Use SCTransform function in Seurat to normalize data. 


R[write to console]: Calculating cell attributes from input UMI matrix: log_umi

R[write to console]: Variance stabilizing transformation of count matrix of size 14262 by 4226

R[write to console]: Model formula is y ~ log_umi

R[write to console]: Get Negative Binomial regression parameters per gene

R[write to console]: Using 2000 genes, 4226 cells



  |======================================================================| 100%


R[write to console]: Found 64 outliers - those will be ignored in fitting/regularization step


R[write to console]: Second step: Get residuals using fitted parameters for 14262 genes



  |======================================================================| 100%


R[write to console]: Computing corrected count matrix for 14262 genes



  |======================================================================| 100%


R[write to console]: Calculating gene attributes

R[write to console]: Wall clock passed: Time difference of 48.41213 secs

R[write to console]: Determine variable features

R[write to console]: Place corrected count matrix in counts slot

R[write to console]: Centering data matrix

  |                                                                            
  |                                                                      |   0%
  |                                                                            
  |====                                                                  |   5%
  |                                                                            
  |=======                                                               |  10%
  |                                                                            
  |==========                                                            |  15%
  |                                                                         

## Use spark.test function in SPARK package to select spatially variable genes. 
## ===== SPARK INPUT INFORMATION ==== 
## number of total samples:  4226 
## number of total features:  3974 
## number of adjusted covariates:  0 
# fitting normalized count-based spatial model under the null hypothesis ... 
## testing Gaussian kernel: 1...
## testing Periodic kernel: 1...
## testing Gaussian kernel: 2...
## testing Periodic kernel: 2...
## testing Gaussian kernel: 3...
## testing Periodic kernel: 3...
## testing Gaussian kernel: 4...
## testing Periodic kernel: 4...
## testing Gaussian kernel: 5...
## testing Periodic kernel: 5...
## Identified  1288  spatial genes through spark.test function. 
The  number of significant spatial genes is less than the specified number of spatial genes. 
## Using  1288  significant spatially variable genes. 
## Selected kernel type is:  gaussian  
## The bandwidth is:  0.0387322493431883  
## Calculating kernel matrix
## Finished calculating kernel matrix

<rpy2.robjects.methods.RS4 object at 0x7fdb504eb3c0> [RTYPES.S4SXP]
R classes: ('SpatialPCA',)

In [53]:
robjects.r('write.csv(as.data.frame(LIBD@SpatialPCs), file = "{}")'.format("test.csv"))

In [3]:
datas = sc.read_h5ad("./datas/nature/151507_preprocessed.h5")
datas

AnnData object with n_obs × n_vars = 4220 × 9948
    obs: 'in_tissue', 'array_row', 'array_col', 'imagerow', 'imagecol', 'sum_umi', 'sum_gene', 'subject', 'position', 'replicate', 'discard', 'cell_count', 'layer_guess', 'layer_guess_reordered', 'layer_guess_reordered_short', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_50_genes', 'pct_counts_in_top_100_genes', 'pct_counts_in_top_200_genes', 'pct_counts_in_top_500_genes', 'n_counts'
    var: 'gene_ids', 'feature_types', 'genome', 'n_cells_by_counts', 'mean_counts', 'log1p_mean_counts', 'pct_dropout_by_counts', 'total_counts', 'log1p_total_counts', 'n_counts'
    uns: 'layer_guess_reordered_colors'
    obsm: 'spatial'

In [19]:
datas = pd.read_csv("./my_list.csv")
datas=datas.set_index(datas.columns[0]).T
datas

Unnamed: 0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20
AAACAACGAATAGTTC-1,1.839585,-0.056076,0.119012,2.259882,-0.725110,4.093265,1.573622,-1.778798,-0.037971,-2.481191,-0.346664,0.488325,-0.090961,0.254027,0.170660,-0.437685,-1.593910,1.668237,-0.665730,0.467312
AAACAAGTATCTCCCA-1,-4.201764,-0.170601,-0.819079,-0.686475,1.279149,-1.377460,1.111393,0.318332,-0.016962,0.454175,0.764330,0.026984,0.718653,-0.332289,-0.268545,-0.011880,0.155666,0.085920,0.066624,0.266871
AAACAATCTACTAGCA-1,8.301020,8.129599,-2.846293,-3.876941,-1.510169,0.406868,-0.511169,-0.609171,1.509160,1.234912,2.693397,-0.351943,0.188804,-0.748378,-0.216233,0.596782,0.166709,0.001538,-0.543832,1.452704
AAACACCAATAACTGC-1,15.506694,-9.032607,5.197555,-4.409749,2.281063,-1.550231,-0.261578,1.772977,0.224246,-0.274725,-0.701496,-1.173895,1.063487,0.523630,0.463621,0.154000,-1.093090,-0.941864,1.582432,0.393645
AAACAGCTTTCAGAAG-1,-2.607821,-1.672040,-0.047325,-0.060443,-3.080712,1.288785,1.136652,2.336629,-2.891868,2.962956,0.465117,-0.953662,-0.387020,0.941422,1.582869,0.892871,0.777105,0.299855,0.632999,-0.688456
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTGTTGTGTGTCAAGA-1,-3.570083,2.295234,0.850403,-1.855247,1.060709,-0.193066,-0.799369,-0.620065,0.082121,-0.674397,-0.035970,0.044762,-0.630693,-0.392696,0.485399,-0.082156,-0.230145,0.279488,0.728626,-0.108202
TTGTTTCACATCCAGG-1,0.154293,-3.614489,1.909944,-0.325207,-2.360692,-0.082183,-0.339690,-1.573506,-1.503427,1.239652,-0.406086,0.986256,-1.195355,-0.468261,0.300783,-0.376656,0.185939,-0.003332,0.433184,-0.166631
TTGTTTCATTAGTCTA-1,10.263456,-7.831578,1.197190,2.795934,2.954854,2.208829,-0.287341,-1.218083,-2.846245,-0.242626,1.136412,2.561909,2.410297,-2.321355,-1.923278,1.137040,1.663151,1.269623,-1.126727,1.399827
TTGTTTCCATACAACT-1,-1.416969,-2.258090,-0.229993,-0.779353,-1.631411,0.862991,-1.129650,0.348168,0.290358,0.698683,-0.660364,-0.642317,0.723156,-0.166788,1.208592,0.245354,0.020271,-0.538028,-0.094238,-0.850288


In [14]:
datas.obs.columns

Index(['in_tissue', 'array_row', 'array_col', 'imagerow', 'imagecol',
       'sum_umi', 'sum_gene', 'subject', 'position', 'replicate', 'discard',
       'cell_count', 'layer_guess', 'layer_guess_reordered',
       'layer_guess_reordered_short', 'n_genes_by_counts',
       'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts',
       'pct_counts_in_top_50_genes', 'pct_counts_in_top_100_genes',
       'pct_counts_in_top_200_genes', 'pct_counts_in_top_500_genes',
       'n_counts'],
      dtype='object')

count    4220.0
mean        0.0
std         0.0
min         0.0
25%         0.0
50%         0.0
75%         0.0
max         0.0
Name: position, dtype: float64